In [1]:
import sys
print(sys.executable)
print(sys.path)

c:\Users\N1259534\AppData\Local\anaconda3\python.exe
['c:\\Users\\N1259534\\OneDrive - Nottingham Trent University\\Desktop\\Projects\\QAOA_simulation_LV\\QAOA_mtrix_simulation', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\python311.zip', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\DLLs', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\Lib', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3', '', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\Lib\\site-packages', '__editable__.qokit-0.1.4.finder.__path_hook__', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\N1259534\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin']


In [ ]:
import os
if 'VIRTUAL_ENV' in os.environ:
    print("Running in a virtual environment:", os.environ['VIRTUAL_ENV'])
else:
    print("Not running in a virtual environment")

In [ ]:
sys.path.append(r"C:\Users\N1259534\Desktop\Projects\QAOA_simulation_LV\QAOA_mtrix_simulation")

In [ ]:

import numpy as np
from itertools import combinations
import scipy
import networkx as nx
import matplotlib.pyplot as plt

from QAOA_objective import get_qaoa_objective


In [2]:
import numba.cuda
print(numba.cuda.is_available())

ImportError: Numba needs NumPy 1.25 or less

In [ ]:
#Step 1: Generate terms

N = 10
np.random.seed(1)
terms = [(np.random.normal(), spin_pair) for spin_pair in combinations(range(N), r=2)]
print(terms)

In [ ]:
G = nx.Graph()
for weight, (node1, node2) in terms:
    G.add_edge(node1, node2, wight=round(weight, 2))
    
plt.figure(figsize=(3, 1.5))
pos = nx.spring_layout(G)  # Layout for visualization
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=500, font_size=12)


plt.show()

In [ ]:
import cupy as cp


In [ ]:
# Step 2: get QAOA objective
# Get objective
p =4
f = get_qaoa_objective(N, terms=terms, parameterization='theta', simulator='gpu')

In [ ]:
# Step 3: Optimize QAOA parameters
p = 4
initial_gamma = -1*np.linspace(0, 1, p)
initial_beta = np.linspace(1, 0, p)

res = scipy.optimize.minimize(f, np.hstack([initial_gamma, initial_beta]), method='COBYLA', options={'rhobeg': 0.01})

In [ ]:
nvcc --version

In [ ]:
print(f"Expected QAOA solution quality: {res.fun:.5f}")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(-res.x[:p], label=r"$\gamma$")
plt.plot(res.x[p:], label=r"$\beta$")
plt.title('Optimized QAOA parameters')
plt.legend()

In [ ]:
def f_from_terms_ground_truth(s):
    """ground truth for debugging
    """
    out = 0
    for coeff, (i, j) in terms:
        out += coeff * s[i] * s[j]
    return out

In [ ]:
print(f"True minimum: {brute_force(f_from_terms_ground_truth, N, minimize=True)[0]:.5f}")
